## Day 24 - Colliding hailstones

**Part 1: Disregarding z-axis, how many hailstones will collide between with an X and Y position each at least 200000000000000 and at most 400000000000000.**

In [9]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['19, 13, 30 @ -2,  1, -2',
 '18, 19, 22 @ -1, -1, -2',
 '20, 25, 34 @ -2, -2, -4',
 '12, 31, 28 @ -1, -2, -1',
 '20, 19, 15 @  1, -5, -3']

In [10]:
def intersection_time2(x1, x2, y1, y2, vx1, vx2, vy1, vy2) -> float:
    return (x2 - x1 + (vx1/vy1)*(y1-y2))/((vx1*vy2)/vy1 - vx2)

In [11]:
def position_at_time2(x2, y2, vx2, vy2, t2):
    return (x2 + vx2*t2, y2 + vy2*t2)

In [12]:
def t1_from_xval1(xs, vx1, xval1):
    return (xval1 - xs)/vx1

In [13]:
from itertools import combinations

def part1(lines: list[str], example: bool = False) -> int:
    lower_bound = 7 if example else 200000000000000
    upper_bound = 27 if example else 400000000000000

    hailstones = [
        tuple(map(int,
                  (line.split(",")[0], line.split(", ")[1], line.split("@ ")[-1].split(",")[0], line.split("@ ")[-1].split(", ")[1])
                  ))
        for line in lines
    ]  # (X, Y, vX, vY)

    found = 0

    for (x1, y1, vx1, vy1), (x2, y2, vx2, vy2) in combinations(hailstones, r=2):
        try:
            t2 = intersection_time2(x1, x2, y1, y2, vx1, vx2, vy1, vy2)
            xval2, yval2 = position_at_time2(x2, y2, vx2, vy2, t2)
            t1 = t1_from_xval1(x1, vx1, xval2)

        except:
            continue

        if (
            lower_bound <= xval2 <= upper_bound
            and lower_bound <= yval2 <= upper_bound
            and t2 >= 0
            and t1 >= 0
        ):
            found += 1

    return found

assert part1(example_lines, example=True) == 2
part1(input_lines, example=False)  # with some Algebra help from Kate!

14799